<a href="https://colab.research.google.com/github/wanla9999/guest/blob/master/aug_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("GPU是否可用：",tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU是否可用： True


In [2]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
"""#cic_2017
label_name = "label"
extra_label_name1 = ' Label'
extra_label_name2 = 'old_label'
path = "/content/Augmented_CIC-IDS2017.csv"
df = pd.read_csv(path)
#df = dd.read_csv(path)
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])
# X will contain all features except the label columns
X = df.drop(['label',' Label','old_label','label_encoded'], axis=1)
print(X.head())
# Y will be the encoded labels
y = df['label_encoded']
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)"""


'#cic_2017\nlabel_name = "label"\nextra_label_name1 = \' Label\'\nextra_label_name2 = \'old_label\'\npath = "/content/Augmented_CIC-IDS2017.csv"\ndf = pd.read_csv(path)\n#df = dd.read_csv(path)\nlabel_encoder = LabelEncoder()\ndf["label_encoded"] = label_encoder.fit_transform(df["label"])\n# X will contain all features except the label columns\nX = df.drop([\'label\',\' Label\',\'old_label\',\'label_encoded\'], axis=1)\nprint(X.head())\n# Y will be the encoded labels\ny = df[\'label_encoded\']\nprint(y.head())\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)'

In [8]:
def process_data(path,label_name,extra_label_name1,extra_label_name2):
    df = pd.read_csv(path)
    #df = df.dropna(axis=0,how='any')# 删除全为 NaN 的行
    #df = dd.read_csv(path)
    label_encoder = LabelEncoder()
    df["label_encoded"] = label_encoder.fit_transform(df[label_name])
    # X will contain all features except the label columns
    if label_name == 'attack_cat':
        X = df.drop([label_name,'label_encoded'], axis=1)
    elif label_name == 'label':
        X = df.drop([label_name,extra_label_name1,extra_label_name2,'label_encoded'], axis=1)

    # Identify categorical columns
    categorical_cols = X.select_dtypes(include=['object']).columns

    # Apply one-hot encoding to categorical columns
    X = pd.get_dummies(X, columns=categorical_cols, dummy_na=False)

    print(X.head())
    # Y will be the encoded labels
    y = df['label_encoded']
    print(y.head())
    #X, y = make_classification(n_samples=5000, n_features=50, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test
#cic_2017
#label_name = "label"
#extra_label_name1 = ' Label'
#extra_label_name2 = 'old_label'
#path = "/content/Augmented_CIC-IDS2017.csv"
#unsw_nb15
label_name = "attack_cat"
extra_label_name1 = ''
extra_label_name2 = ''
path = "/content/Augmented-UNSW_NB15.csv"
X_train, X_test, y_train, y_test = process_data(path, label_name, extra_label_name1, extra_label_name2)
#numpy array

   id       dur  spkts  dpkts  sbytes  dbytes       rate  sttl  dttl  \
0   1  0.121478      6      4     258     172  74.087490   252   254   
1   2  0.649902     14     38     734   42014  78.473372    62   252   
2   3  1.623129      8     16     364   13186  14.170161    62   252   
3   4  1.681642     12     12     628     770  13.677108    62   252   
4   5  0.449454     10      6     534     268  33.373826   254   252   

          sload  ...  state_CLO  state_CON  state_ECO  state_FIN  state_INT  \
0  14158.942380  ...      False      False      False       True      False   
1   8395.112305  ...      False      False      False       True      False   
2   1572.271851  ...      False      False      False       True      False   
3   2740.178955  ...      False      False      False       True      False   
4   8561.499023  ...      False      False      False       True      False   

   state_PAR  state_REQ  state_RST  state_URN  state_no  
0      False      False      False

In [9]:
# K近邻
model1 = KNeighborsClassifier(
    n_neighbors=10,
    n_jobs=-1,  # 使用所有CPU核心
    leaf_size=30  # 调整树的大小影响并行效率
)
# 决策树
model2 = DecisionTreeClassifier(random_state=77)
# 随机森林
model3 = RandomForestClassifier(
    n_estimators=1000,
    max_features='sqrt',
    max_depth=15,
    min_samples_leaf=3,
    n_jobs=-1,
    random_state=42
)
# 神经网络
model4 = MLPClassifier(hidden_layer_sizes=(16, 8), random_state=77, max_iter=10000)

model_list = [model1, model2, model3, model4]
model_name = ['K近邻', '决策树', '随机森林', '神经网络']


def evaluation(y_test, y_predict):
    print("------evaluation------")
    accuracy = classification_report(y_test, y_predict, output_dict=True)['accuracy']
    s = classification_report(y_test, y_predict, output_dict=True)['weighted avg']
    precision = s['precision']
    recall = s['recall']
    f1_score = s['f1-score']
    # kappa=cohen_kappa_score(y_test, y_predict)
    return accuracy, precision, recall, f1_score  # , kappa


df_eval=pd.DataFrame(columns=['Accuracy','Precision','Recall','F1_score'])
for i in range(4):
  print(i)
  model_C=model_list[i]
  name=model_name[i]
  model_C.fit(X_train, y_train)
  print("----fit done-----")
  pred=model_C.predict(X_test)
  #s=classification_report(y_test, pred)
  s=evaluation(y_test,pred)
  df_eval.loc[name,:]=list(s)
  #path = 'DataSets/output.csv'
  #df_eval.to_csv(path, index=True)
print(df_eval.head())

0
----fit done-----
------evaluation------
1
----fit done-----
------evaluation------
2
----fit done-----
------evaluation------
3
----fit done-----
------evaluation------
      Accuracy Precision    Recall  F1_score
K近邻   0.724432   0.73476  0.724432  0.710645
决策树   0.871776  0.873765  0.871776  0.872734
随机森林  0.863683  0.877269  0.863683  0.836781
神经网络  0.470514   0.26169  0.470514  0.301309
